## Develop a recommendation system that suggests movies to users based on their viewing history,preferences, or ratings using machine learning techniques.

In [2]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
ratings = pd.read_csv("C:/Users/vedan/OneDrive/Documents/cogenetix/task 9/ml-100k/u.data", sep = "\t",
                     names = ["userId", "movieId", "rating", "timesstamp"])

In [4]:
movies = pd.read_csv("C:/Users/vedan/OneDrive/Documents/cogenetix/task 9/ml-100k/u.item", sep = "|",
                    encoding = "latin-1", header = None)
movies = movies[[0, 1]]
movies.columns = ["movieId", "title"]

In [5]:
df = ratings.merge(movies, on="movieId")
df.head()

,userId,movieId,rating,timesstamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [6]:
# EDA (FAST)
# 🔹 Top Rated Movies
df.groupby("title")["rating"].mean().sort_values(ascending=False).head(10)

title
They Made Me a Criminal (1939)                       5.0
Marlene Dietrich: Shadow and Light (1996)            5.0
Saint of Fort Washington, The (1993)                 5.0
Someone Else's America (1995)                        5.0
Star Kid (1997)                                      5.0
Great Day in Harlem, A (1994)                        5.0
Aiqing wansui (1994)                                 5.0
Santa with Muscles (1996)                            5.0
Prefontaine (1997)                                   5.0
Entertaining Angels: The Dorothy Day Story (1996)    5.0
Name: rating, dtype: float64

In [7]:
# Most Rated Movies
df.groupby("title")["rating"].count().sort_values(ascending=False).head(10)

title
Star Wars (1977)                 583
Contact (1997)                   509
Fargo (1996)                     508
Return of the Jedi (1983)        507
Liar Liar (1997)                 485
English Patient, The (1996)      481
Scream (1996)                    478
Toy Story (1995)                 452
Air Force One (1997)             431
Independence Day (ID4) (1996)    429
Name: rating, dtype: int64

In [11]:
# User–Movie Matrix
user_movie = df.pivot_table(
    index="userId",
    columns="title",
    values="rating"
)

In [12]:
# Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity

movie_similarity = cosine_similarity(
    user_movie.fillna(0).T
)

In [13]:
# Similarity DataFrame
similarity_df = pd.DataFrame(
    movie_similarity,
    index=user_movie.columns,
    columns=user_movie.columns
)

In [14]:
# Recommendation Function

def recommend(movie, n=5):
    return similarity_df[movie].sort_values(ascending=False)[1:n+1]


In [15]:
recommend("Star Wars (1977)")

title
Return of the Jedi (1983)          0.884476
Raiders of the Lost Ark (1981)     0.764885
Empire Strikes Back, The (1980)    0.749819
Toy Story (1995)                   0.734572
Godfather, The (1972)              0.697332
Name: Star Wars (1977), dtype: float64

In [17]:
# Evaluation (RMSE)

from sklearn.metrics import mean_squared_error


rmse = np.sqrt(mean_squared_error(
    df["rating"],
    [df["rating"].mean()] * len(df)
))

rmse

1.1256679707622492

In [18]:
import joblib

joblib.dump(similarity_df, "movie_similarity.pkl")
joblib.dump(user_movie.columns, "movie_titles.pkl")

['movie_titles.pkl']